# Semana 4 - Relatórios

In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from datetime import datetime

In [4]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master('local[2]')
                     .appName('Relatorios')
        )

# Importando arquivos parquet

In [5]:
df_airports_qa = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/airports_qa.parquet")

df_planes_qa = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/planes_qa.parquet")

df_flights_qa = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/flights_qa.parquet")

df_airports_proc = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/airports_proc.parquet")

df_planes_proc = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/planes_proc.parquet")

df_flights_proc = spark.getOrCreate().read.parquet("C:/Users/tsaheki/Documents/Treinamento PySpark - Relatorios/flights_proc.parquet")


#  PERGUNTAS DE QUALIDADE

# Pergunta 1

In [6]:
df_airports_origin = df_airports_qa.select([F.col(c).alias(c+"_airports_origin") for c in df_airports_qa.columns])
df_airports_dest = df_airports_qa.select([F.col(c).alias(c+"_airports_dest") for c in df_airports_qa.columns])
df_planes = df_planes_qa.select([F.col(c).alias(c+"_planes") for c in df_planes_qa.columns])
df_flights = df_flights_qa.select([F.col(c).alias(c+"_flights") for c in df_flights_qa.columns])

In [7]:
df_unificado = df_flights.join(df_airports_origin, df_flights.origin_flights == df_airports_origin.faa_airports_origin, 'left')

df_unificado = df_unificado.join(df_airports_dest, df_unificado.dest_flights == df_airports_dest.faa_airports_dest, 'left')

df_unificado = df_unificado.join(df_planes, df_unificado.tailnum_flights == df_planes.tailnum_planes, 'left')

df_unificado = df_unificado.select(df_unificado.colRegex("`^qa_.*`"))

df_unificado.toPandas()

,qa_year_month_day_flights,qa_hour_minute_flights,qa_dep_arr_flights,qa_dep_arr_delay_flights,qa_carrier_flights,qa_tailnum_flights,qa_flight_flights,qa_origin_dest_flights,qa_air_time_flights,qa_distance_flights,...,qa_dst_airports_dest,qa_tailnum_planes,qa_year_planes,qa_type_planes,qa_manufacturer_planes,qa_model_planes,qa_engines_planes,qa_seats_planes,qa_speed_planes,qa_engine_planes
0,None,None,None,None,None,None,None,None,None,None,...,None,S,None,None,None,None,None,None,M,None
1,None,None,None,None,None,None,F,None,None,None,...,None,S,None,None,None,None,None,None,M,None
2,None,None,None,None,None,None,F,None,None,None,...,None,S,None,None,None,None,None,None,M,None
3,None,None,None,None,None,None,F,None,None,None,...,None,S,None,None,None,None,None,None,M,None
4,None,None,None,None,None,None,F,None,None,None,...,None,S,None,None,None,None,None,None,M,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,None,None,None,None,None,None,None,None,None,...,None,S,None,None,C,None,None,None,M,None
9996,None,None,None,None,None,F,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9997,None,None,None,None,None,None,F,None,None,None,...,None,S,None,None,None,None,None,None,M,None
9998,None,None,None,None,None,None,None,None,None,None,...,None,S,None,None,None,None,None,None,M,None


# Pergunta 2

In [8]:
names = df_unificado.schema.names

for c in names:
    df_unificado = df_unificado.withColumn(c, (F.when((F.col(c).startswith("M")) ,"M")
                                                .when((F.col(c).startswith("F")) ,"F")
                                                .when((F.col(c).startswith("I")) ,"I")
                                                .when((F.col(c).startswith("S")) ,"S")
                                                .when((F.col(c).startswith("T")) ,"T")
                                                .otherwise(F.col(c))
                                      )
                                  )    
    df_unificado.filter((F.col(c).isNotNull()) & (F.col(c).isin("M","F","I","S","T"))).select(F.col(c)).groupBy(F.col(c).alias("Issues")).agg(F.count(F.col(c)).cast('int').alias(c)).orderBy(F.col(c)).show()
    

+------+-------------------------+
|Issues|qa_year_month_day_flights|
+------+-------------------------+
+------+-------------------------+

+------+----------------------+
|Issues|qa_hour_minute_flights|
+------+----------------------+
|     M|                    48|
+------+----------------------+

+------+------------------+
|Issues|qa_dep_arr_flights|
+------+------------------+
|     M|                55|
|     F|               241|
+------+------------------+

+------+------------------------+
|Issues|qa_dep_arr_delay_flights|
+------+------------------------+
|     M|                      75|
+------+------------------------+

+------+------------------+
|Issues|qa_carrier_flights|
+------+------------------+
+------+------------------+

+------+------------------+
|Issues|qa_tailnum_flights|
+------+------------------+
|     S|                14|
|     F|              1024|
+------+------------------+

+------+-----------------+
|Issues|qa_flight_flights|
+------+--------------

## Pergunta 2 (TRANSPOR DADOS)

In [ ]:
columns = ['Var', 'F', 'I', 'M', 'S', 'T']
Vals = [("Col1", 0, 0, 0, 0, 0)]

df_transposed = spark.getOrCreate().createDataFrame(Vals, columns)
df_transposed = df_transposed.filter(F.col("Var") != "Col1")

names = df_final.schema.names

for c in names:
    df_final = df_final.withColumn(c, (F.when((F.col(c).startswith("M")) ,"M")
                                                                .when((F.col(c).startswith("F")) ,"F")
                                                                .when((F.col(c).startswith("I")) ,"I")
                                                                .when((F.col(c).startswith("S")) ,"S")
                                                                .when((F.col(c).startswith("T")) ,"T")
                                                                .otherwise(F.col(c))
                                      )
                                  )
    df = df_final.filter((F.col(c).isNotNull()) & (F.col(c).isin("M","F","I","S","T"))).select(F.col(c)).groupBy(F.col(c).alias("Issues_")).agg(F.count(F.col(c)).cast('int').alias(c)).orderBy(F.col(c))
    df = (df.withColumn("F", (F.when((F.col("Issues_").startswith("F")), F.col(c)+0).otherwise(0)))
            .withColumn("I", (F.when((F.col("Issues_").startswith("I")), F.col(c)+0).otherwise(0)))
            .withColumn("M", (F.when((F.col("Issues_").startswith("M")), F.col(c)+0).otherwise(0)))
            .withColumn("S", (F.when((F.col("Issues_").startswith("S")), F.col(c)+0).otherwise(0)))
            .withColumn("T", (F.when((F.col("Issues_").startswith("T")), F.col(c)+0).otherwise(0)))
            .withColumn("Var", (F.when((F.col("Issues_").isNotNull()), c)))
         )
    
    df = df.select("Var", "F", "I", "M", "S", "T")
    
    df_transposed = df_transposed.union(df)

# Pergunta 3

In [9]:
names = df_unificado.schema.names

for c in names:
    df_unificado = df_unificado.withColumn(c, (F.when((F.col(c).startswith("M")) ,"M")
                                                .otherwise(F.col(c))
                                      )
                                  )    
    df_unificado.filter((F.col(c).isNotNull()) & (F.col(c) == "M")).select(F.col(c)).groupBy(F.col(c).alias("Issues")).agg(F.count(F.col(c)).cast('int').alias(c)).orderBy(F.col(c)).show()
   


+------+-------------------------+
|Issues|qa_year_month_day_flights|
+------+-------------------------+
+------+-------------------------+

+------+----------------------+
|Issues|qa_hour_minute_flights|
+------+----------------------+
|     M|                    48|
+------+----------------------+

+------+------------------+
|Issues|qa_dep_arr_flights|
+------+------------------+
|     M|                55|
+------+------------------+

+------+------------------------+
|Issues|qa_dep_arr_delay_flights|
+------+------------------------+
|     M|                      75|
+------+------------------------+

+------+------------------+
|Issues|qa_carrier_flights|
+------+------------------+
+------+------------------+

+------+------------------+
|Issues|qa_tailnum_flights|
+------+------------------+
+------+------------------+

+------+-----------------+
|Issues|qa_flight_flights|
+------+-----------------+
+------+-----------------+

+------+----------------------+
|Issues|qa_origin_d

# Pergunta 3 (Resposta do DF TRANSPOSE)

In [ ]:
(df_transposed.groupBy("Var")
              .agg(F.max('M').alias('Max_M'))
              .agg(F.max(F.struct(F.col('Max_M'), F.col('Var'))).alias('Max_Var'))
              .select('Max_Var.Var', 'Max_Var.Max_M').show())

# Pergunta 4

In [10]:
names = df_unificado.schema.names

for c in names:
    df_unificado = df_unificado.withColumn(c, (F.when((F.col(c).startswith("F")) ,"F")
                                                .otherwise(F.col(c))
                                      )
                                  )    
    df_unificado.filter((F.col(c).isNotNull()) & (F.col(c) == "F")).select(F.col(c)).groupBy(F.col(c).alias("Issues")).agg(F.count(F.col(c)).cast('int').alias(c)).orderBy(F.col(c)).show()
   


+------+-------------------------+
|Issues|qa_year_month_day_flights|
+------+-------------------------+
+------+-------------------------+

+------+----------------------+
|Issues|qa_hour_minute_flights|
+------+----------------------+
+------+----------------------+

+------+------------------+
|Issues|qa_dep_arr_flights|
+------+------------------+
|     F|               241|
+------+------------------+

+------+------------------------+
|Issues|qa_dep_arr_delay_flights|
+------+------------------------+
+------+------------------------+

+------+------------------+
|Issues|qa_carrier_flights|
+------+------------------+
+------+------------------+

+------+------------------+
|Issues|qa_tailnum_flights|
+------+------------------+
|     F|              1024|
+------+------------------+

+------+-----------------+
|Issues|qa_flight_flights|
+------+-----------------+
|     F|             6158|
+------+-----------------+

+------+----------------------+
|Issues|qa_origin_dest_flights

# Pergunta 4 (Resposta do DF TRANSPOSE)

In [ ]:
(df_transposed.groupBy("Var")
              .agg(F.max('F').alias('Max_F'))
              .agg(F.max(F.struct(F.col('Max_F'), F.col('Var'))).alias('Max_Var'))
              .select('Max_Var.Var', 'Max_Var.Max_F').show())

# Pergunta 5

In [11]:
names = df_unificado.schema.names

for c in names:
    df_unificado = df_unificado.withColumn(c, (F.when((F.col(c).startswith("I")) ,"I")
                                                .otherwise(F.col(c))
                                      )
                                  )    
    df_unificado.filter((F.col(c).isNotNull()) & (F.col(c) == "I")).select(F.col(c)).groupBy(F.col(c).alias("Issues")).agg(F.count(F.col(c)).cast('int').alias(c)).orderBy(F.col(c)).show()
   

+------+-------------------------+
|Issues|qa_year_month_day_flights|
+------+-------------------------+
+------+-------------------------+

+------+----------------------+
|Issues|qa_hour_minute_flights|
+------+----------------------+
+------+----------------------+

+------+------------------+
|Issues|qa_dep_arr_flights|
+------+------------------+
+------+------------------+

+------+------------------------+
|Issues|qa_dep_arr_delay_flights|
+------+------------------------+
+------+------------------------+

+------+------------------+
|Issues|qa_carrier_flights|
+------+------------------+
+------+------------------+

+------+------------------+
|Issues|qa_tailnum_flights|
+------+------------------+
+------+------------------+

+------+-----------------+
|Issues|qa_flight_flights|
+------+-----------------+
+------+-----------------+

+------+----------------------+
|Issues|qa_origin_dest_flights|
+------+----------------------+
+------+----------------------+

+------+--------

# Pergunta 5 (Resposta do DF TRANSPOSE)

In [ ]:
(df_transposed.groupBy("Var")
              .agg(F.max('I').alias('Max_I'))
              .agg(F.max(F.struct(F.col('Max_I'), F.col('Var'))).alias('Max_Var'))
              .select('Max_Var.Var', 'Max_Var.Max_I').show())

# PERGUNTAS DE NEGÓCIO

# Pergunta 1

In [12]:
df_airports_origin = df_airports_proc.select([F.col(c).alias(c+"_airports_origin") for c in df_airports_proc.columns])
df_airports_dest = df_airports_proc.select([F.col(c).alias(c+"_airports_dest") for c in df_airports_proc.columns])
df_planes = df_planes_proc.select([F.col(c).alias(c+"_planes") for c in df_planes_proc.columns])
df_flights = df_flights_proc.select([F.col(c).alias(c+"_flights") for c in df_flights_proc.columns])

In [13]:
df_unificado_proc = df_flights.join(df_airports_origin, df_flights.origin_flights == df_airports_origin.faa_airports_origin, 'left')

df_unificado_proc = df_unificado_proc.join(df_airports_dest, df_unificado_proc.dest_flights == df_airports_dest.faa_airports_dest, 'left')

df_unificado_proc = df_unificado_proc.join(df_planes, df_unificado_proc.tailnum_flights == df_planes.tailnum_planes, 'left')


In [14]:
# Criando view

df_unificado_proc.createOrReplaceTempView('unificado_proc')

# Pergunta 2

In [15]:
(df_unificado_proc.drop_duplicates(['faa_airports_dest']).select('faa_airports_dest', 'region_airports_dest')).groupBy('region_airports_dest').count().show()

+--------------------+-----+
|region_airports_dest|count|
+--------------------+-----+
|              ALASKA|    9|
|       MAINLAND-EAST|   24|
|       MAINLAND-WEST|   36|
+--------------------+-----+



# Pergunta 3

In [16]:
df_unificado_proc.agg(F.max((F.col('alt_airports_origin'))-(F.col('alt_airports_dest')))).show()

+----------------------------------------------+
|max((alt_airports_origin - alt_airports_dest))|
+----------------------------------------------+
|                                           429|
+----------------------------------------------+



# Pergunta 4

In [17]:
(df_unificado_proc.filter(F.col('arr_delay_flights')>0)).agg(F.avg(F.col('arr_delay_flights'))).show()

(df_unificado_proc.filter(F.col('dep_delay_flights')>0)).agg(F.avg(F.col('dep_delay_flights'))).show()

+----------------------+
|avg(arr_delay_flights)|
+----------------------+
|      24.6496644295302|
+----------------------+

+----------------------+
|avg(dep_delay_flights)|
+----------------------+
|      25.5242774566474|
+----------------------+



# Pergunta 5

In [18]:
(df_unificado_proc.filter(F.col('arr_delay_flights')>0)).groupBy('region_airports_dest').avg('arr_delay_flights').show()

#.agg(F.avg(F.col('arr_delay_flights'))).show()




+--------------------+----------------------+
|region_airports_dest|avg(arr_delay_flights)|
+--------------------+----------------------+
|              ALASKA|    22.210796915167094|
|       MAINLAND-EAST|    28.529850746268657|
|       MAINLAND-WEST|     23.79225908372828|
+--------------------+----------------------+



# Pergunta 6

In [19]:
(df_unificado_proc.filter(F.col('arr_delay_flights')>0)).groupBy(F.year(F.to_timestamp('dep_datetime_flights'))).sum('arr_delay_flights').show()

+----------------------------------------+----------------------+
|year(to_timestamp(dep_datetime_flights))|sum(arr_delay_flights)|
+----------------------------------------+----------------------+
|                                    2014|                 91820|
+----------------------------------------+----------------------+



# Pergunta 7

In [20]:
(df_unificado_proc.filter(F.col('arr_delay_flights')>0)).groupBy(F.year(F.to_timestamp('dep_datetime_flights')), 'region_airports_dest').sum('arr_delay_flights').show()

+----------------------------------------+--------------------+----------------------+
|year(to_timestamp(dep_datetime_flights))|region_airports_dest|sum(arr_delay_flights)|
+----------------------------------------+--------------------+----------------------+
|                                    2014|              ALASKA|                  8640|
|                                    2014|       MAINLAND-EAST|                 22938|
|                                    2014|       MAINLAND-WEST|                 60242|
+----------------------------------------+--------------------+----------------------+



# Pergunta 8

In [21]:
df_unificado_proc.agg(F.avg(F.col('air_time_flights'))).show()

+---------------------+
|avg(air_time_flights)|
+---------------------+
|               152.43|
+---------------------+



# Pergunta 9

In [22]:
df_unificado_proc.groupBy('region_airports_dest').avg('air_time_flights').show()

+--------------------+---------------------+
|region_airports_dest|avg(air_time_flights)|
+--------------------+---------------------+
|              ALASKA|   227.35474308300394|
|       MAINLAND-EAST|   236.58582089552237|
|       MAINLAND-WEST|   114.98787258912917|
+--------------------+---------------------+



# Pergunta 10

In [23]:
df_unificado_proc.groupBy('faa_airports_origin','faa_airports_dest').avg('air_time_flights').show()

+-------------------+-----------------+---------------------+
|faa_airports_origin|faa_airports_dest|avg(air_time_flights)|
+-------------------+-----------------+---------------------+
|                SEA|              RNO|                 76.0|
|                SEA|              DTW|                219.0|
|                SEA|              CLE|                233.0|
|                SEA|              LAX|                126.0|
|                PDX|              SEA|                 34.0|
|                SEA|              BLI|                 29.0|
|                PDX|              IAH|                213.0|
|                PDX|              PHX|                130.0|
|                SEA|              SLC|                 88.0|
|                SEA|              SBA|                118.0|
|                SEA|              BWI|                269.0|
|                PDX|              IAD|                267.0|
|                PDX|              SFO|                 85.0|
|       

# Pergunta 11 

In [24]:
df_unificado_proc.groupBy(F.year(F.to_timestamp('dep_datetime_flights'))).sum('air_time_flights').show()

+----------------------------------------+---------------------+
|year(to_timestamp(dep_datetime_flights))|sum(air_time_flights)|
+----------------------------------------+---------------------+
|                                    2014|              1524300|
+----------------------------------------+---------------------+



# Pergunta 12

In [25]:
df_unificado_proc.groupBy('region_airports_dest').sum('air_time_flights').show()

+--------------------+---------------------+
|region_airports_dest|sum(air_time_flights)|
+--------------------+---------------------+
|              ALASKA|               230083|
|       MAINLAND-EAST|               507240|
|       MAINLAND-WEST|               786977|
+--------------------+---------------------+



# Pergunta 13

In [26]:
df_unificado_proc.agg(F.avg(F.col('distance_flights'))).show()

+---------------------+
|avg(distance_flights)|
+---------------------+
|            1208.1516|
+---------------------+



# Pergunta 14

In [27]:
df_unificado_proc.groupBy('region_airports_dest').avg('distance_flights').show()

+--------------------+---------------------+
|region_airports_dest|avg(distance_flights)|
+--------------------+---------------------+
|              ALASKA|    1741.653162055336|
|       MAINLAND-EAST|   2042.3983208955224|
|       MAINLAND-WEST|    867.9224137931035|
+--------------------+---------------------+



# Pergunta 15

In [28]:
df_unificado_proc.groupBy('faa_airports_origin','faa_airports_dest').avg('distance_flights').show()

+-------------------+-----------------+---------------------+
|faa_airports_origin|faa_airports_dest|avg(distance_flights)|
+-------------------+-----------------+---------------------+
|                SEA|              RNO|                564.0|
|                SEA|              DTW|               1927.0|
|                SEA|              CLE|               2021.0|
|                SEA|              LAX|                954.0|
|                PDX|              SEA|                129.0|
|                SEA|              BLI|                 93.0|
|                PDX|              IAH|               1825.0|
|                PDX|              PHX|               1009.0|
|                SEA|              SLC|                689.0|
|                SEA|              SBA|                908.0|
|                SEA|              BWI|               2335.0|
|                PDX|              IAD|               2327.0|
|                PDX|              SFO|                550.0|
|       

# Pergunta 16

In [29]:
df_unificado_proc.groupBy(F.year(F.to_timestamp('dep_datetime_flights'))).sum('distance_flights').show()

+----------------------------------------+---------------------+
|year(to_timestamp(dep_datetime_flights))|sum(distance_flights)|
+----------------------------------------+---------------------+
|                                    2014|             12081516|
+----------------------------------------+---------------------+



# Pergunta 17

In [30]:
df_unificado_proc.groupBy('region_airports_dest').sum('distance_flights').show()

+--------------------+---------------------+
|region_airports_dest|sum(distance_flights)|
+--------------------+---------------------+
|              ALASKA|              1762553|
|       MAINLAND-EAST|              4378902|
|       MAINLAND-WEST|              5940061|
+--------------------+---------------------+



# Pergunta 18

In [31]:
df_unificado_proc.groupBy('faa_airports_origin','faa_airports_dest').avg('seats_planes').show()

+-------------------+-----------------+------------------+
|faa_airports_origin|faa_airports_dest| avg(seats_planes)|
+-------------------+-----------------+------------------+
|                SEA|              RNO|           141.125|
|                SEA|              DTW|212.42857142857142|
|                SEA|              CLE|             182.0|
|                SEA|              LAX| 154.7111111111111|
|                PDX|              SEA| 64.28169014084507|
|                SEA|              BLI|             163.6|
|                PDX|              IAH|182.63636363636363|
|                PDX|              PHX|195.63768115942028|
|                SEA|              SLC|165.93303571428572|
|                SEA|              SBA|              80.0|
|                SEA|              BWI|151.27586206896552|
|                PDX|              IAD| 187.3913043478261|
|                PDX|              SFO|138.27814569536423|
|                SEA|              KOA|             170.

# Pergunta 19

In [32]:
df_unificado_proc.groupBy(F.year(F.to_timestamp('dep_datetime_flights'))).sum('seats_planes').show()

+----------------------------------------+-----------------+
|year(to_timestamp(dep_datetime_flights))|sum(seats_planes)|
+----------------------------------------+-----------------+
|                                    2014|          1509544|
+----------------------------------------+-----------------+



# Pergunta 20

In [33]:
(df_unificado_proc.groupBy('faa_airports_dest').count()).orderBy(F.col('count').desc()).show(1)

+-----------------+-----+
|faa_airports_dest|count|
+-----------------+-----+
|              SFO|  787|
+-----------------+-----+
only showing top 1 row



# Pergunta 21 

In [34]:
(df_unificado_proc.groupBy('faa_airports_dest').sum('seats_planes')).orderBy(F.col('sum(seats_planes)').desc()).show(1)

+-----------------+-----------------+
|faa_airports_dest|sum(seats_planes)|
+-----------------+-----------------+
|              SFO|           119635|
+-----------------+-----------------+
only showing top 1 row



# Pergunta 22 

In [35]:
(df_unificado_proc.select('faa_airports_origin', 'faa_airports_dest', 'distance_flights').where(F.col('faa_airports_origin')=='PDX')).orderBy(F.col('distance_flights').desc()).show(1)

+-------------------+-----------------+----------------+
|faa_airports_origin|faa_airports_dest|distance_flights|
+-------------------+-----------------+----------------+
|                PDX|              LIH|            2631|
+-------------------+-----------------+----------------+
only showing top 1 row



# Pergunta 23

In [36]:
df_unificado_proc.groupBy(F.month(F.to_timestamp('dep_datetime_flights')), 'faa_airports_dest').count().show()

+-----------------------------------------+-----------------+-----+
|month(to_timestamp(dep_datetime_flights))|faa_airports_dest|count|
+-----------------------------------------+-----------------+-----+
|                                        4|              PHX|   60|
|                                        1|              RDM|    8|
|                                        5|              ONT|    9|
|                                        7|              OMA|    2|
|                                        8|              MDW|   20|
|                                        6|              DEN|   55|
|                                        5|              IAD|    9|
|                                       12|              COS|    2|
|                                       11|              ANC|   34|
|                                        5|              AUS|    4|
|                                        5|              COS|    3|
|                                        2|     

# Pergunta 24

In [37]:
(df_unificado_proc.groupBy('model_planes').count()).orderBy(F.col('count').desc()).show(1)

+------------+-----+
|model_planes|count|
+------------+-----+
|     737-890| 1463|
+------------+-----+
only showing top 1 row



# Pergunta 25

In [47]:
(df_unificado_proc.groupBy('faa_airports_dest', 'model_planes').count()).orderBy(F.col('count').desc(), 'faa_airports_dest', 'model_planes').show()

+-----------------+------------+-----+
|faa_airports_dest|model_planes|count|
+-----------------+------------+-----+
|              DFW|        null|  248|
|              ORD|        null|  168|
|              OAK|     737-7H4|  141|
|              ANC|     737-890|  138|
|              SNA|     737-790|  122|
|              SMF|     737-7H4|  114|
|              LAX|     737-890|  110|
|              LAS|     737-890|  108|
|              SLC|     757-232|  104|
|              DEN|    A320-214|  102|
|              LGB|    A320-232|  102|
|              SAN|     737-890|  102|
|              LAS|     737-7H4|   97|
|              DEN|     737-7H4|   93|
|              PHX|    A320-232|   91|
|              SJC|     737-7H4|   89|
|              PDX|   EMB-120ER|   87|
|              SFO|     737-490|   87|
|              SFO|    A319-131|   86|
|              SFO|    A320-232|   85|
+-----------------+------------+-----+
only showing top 20 rows



# Pergunta 26

In [48]:
df_unificado_proc.groupBy('haul_duration_flights').avg('engines_planes').show()

+---------------------+-------------------+
|haul_duration_flights|avg(engines_planes)|
+---------------------+-------------------+
|            LONG-HAUL|                2.0|
|          MEDIUM-HAUL| 1.9996078431372548|
|           SHORT-HAUL| 1.9992357077346377|
+---------------------+-------------------+



# Pergunta 27

In [50]:
(df_unificado_proc.groupBy('dep_season_flights').count()).orderBy(F.col('count').desc()).show(1)

+------------------+-----+
|dep_season_flights|count|
+------------------+-----+
|            SUMMER| 2918|
+------------------+-----+
only showing top 1 row



# Pergunta 28

In [51]:
(df_unificado_proc.groupBy('faa_airports_dest', 'dep_season_flights').count()).orderBy(F.col('count').desc(), 'faa_airports_dest', 'dep_season_flights').show()

+-----------------+------------------+-----+
|faa_airports_dest|dep_season_flights|count|
+-----------------+------------------+-----+
|              SFO|            SUMMER|  217|
|              SFO|            SPRING|  199|
|              SFO|              FALL|  198|
|              LAX|            SPRING|  176|
|              SFO|            WINTER|  173|
|              DEN|            SUMMER|  172|
|              LAX|            SUMMER|  163|
|              DEN|            SPRING|  151|
|              PHX|            SPRING|  148|
|              ANC|            SUMMER|  145|
|              DEN|              FALL|  145|
|              LAS|            SPRING|  145|
|              LAX|            WINTER|  139|
|              LAX|              FALL|  137|
|              LAS|            SUMMER|  136|
|              PHX|            SUMMER|  134|
|              PHX|            WINTER|  131|
|              LAS|              FALL|  130|
|              ORD|            SUMMER|  129|
|         

# Pergunta 29

In [53]:
(df_unificado_proc.groupBy('dep_delay_category_flights').count()).orderBy(F.col('count').desc()).show(2)

+--------------------------+-----+
|dep_delay_category_flights|count|
+--------------------------+-----+
|               ANTECIPATED| 5894|
|                     MINOR| 3076|
+--------------------------+-----+
only showing top 2 rows



# Pergunta 30

In [56]:
(df_unificado_proc.groupBy('faa_airports_origin', 'faa_airports_dest', 'dep_delay_category_flights').count()).orderBy(F.col('count').desc()).show()

+-------------------+-----------------+--------------------------+-----+
|faa_airports_origin|faa_airports_dest|dep_delay_category_flights|count|
+-------------------+-----------------+--------------------------+-----+
|                SEA|              LAX|               ANTECIPATED|  293|
|                SEA|              SFO|               ANTECIPATED|  245|
|                SEA|              LAS|               ANTECIPATED|  228|
|                SEA|              PHX|               ANTECIPATED|  195|
|                SEA|              ANC|               ANTECIPATED|  192|
|                PDX|              SFO|               ANTECIPATED|  174|
|                SEA|              DEN|               ANTECIPATED|  169|
|                SEA|              ORD|               ANTECIPATED|  167|
|                SEA|              SFO|                     MINOR|  160|
|                SEA|              ANC|                     MINOR|  146|
|                SEA|              DEN|            